In [5]:
import pandas as dd

In [1]:
NUM_OF_JUNCTIONS = 2
ROLLING_WINDOW_LENGTH = 600
TRAVEL_TIME_WINDOW = 600
TRAFFIC_DATASET = "/home/local/ASURITE/kharapan/highway_trafficDataset_2024-3-27_2348hours_1296000steps.csv"
VEHICLE_DATASET = "/home/local/ASURITE/kharapan/highway_vehicleDataset_2024-3-27_2348hours_1296000steps.csv"
LENGTH_OF_SIM = 1_296_000
generate_incidents_csv = True

In [2]:
print(f"Number of junctions considered for dataset prep: {NUM_OF_JUNCTIONS}")
print(f"Rolling window length considered for dataset prep: {ROLLING_WINDOW_LENGTH}")
print(f"Travel time window length considered for dataset prep: {TRAVEL_TIME_WINDOW}")

Number of junctions considered for dataset prep: 2
Rolling window length considered for dataset prep: 600
Travel time window length considered for dataset prep: 600


In [7]:
df_traffic = dd.read_csv(TRAFFIC_DATASET)

/tmp/ipykernel_1965160/3322267942.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traffic = dd.read_csv(TRAFFIC_DATASET)


In [8]:
df_traffic.head()

,step,time_of_day,identified_edge,junction_mean_speed,traffic_count,traffic_occupancy,vehicles_per_lane_1,vehicles_per_lane_0,lane_mean_speed_0,lane_mean_speed_1,incident_edge,incident_start_time,incident_type,accident_label,accident_id,accident_duration,incident_lane
0,0,0,93676377#2-AddedOnRampEdge,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,False,NaN,0,-1
1,0,0,85756582#1-AddedOffRampEdge,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,False,NaN,0,-1
2,1,1,93676377#2-AddedOnRampEdge,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,False,NaN,0,-1
3,1,1,85756582#1-AddedOffRampEdge,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,False,NaN,0,-1
4,2,2,93676377#2-AddedOnRampEdge,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,False,NaN,0,-1


In [9]:
if(generate_incidents_csv):
    print("Generating incidents csv")
    df_incidents_info = df_traffic[df_traffic["accident_label"]][["incident_edge", "incident_type", 'incident_start_time',"accident_duration", "incident_lane"]]

    df_pandas_incidents = df_incidents_info.groupby("incident_start_time").first().reset_index()

    print("Getting head of incidents dataset")

    print(df_pandas_incidents.head())

    # df_pandas_incidents =  df_incidents_info2.compute()

    df_pandas_incidents.to_csv("incidents_data.csv")

    print(f"Number of incidents in this simulation {len(df_pandas_incidents)}")
    print(f"Number of incidents in this simulation with less severe incidents {len(df_pandas_incidents[df_pandas_incidents['incident_type']=='stalled_vehicle'])}")
    print(f"Number of incidents in this simulation with more severe incidents {len(df_pandas_incidents[df_pandas_incidents['incident_type']=='multi_vehicle_collision'])}")

    print(f"Input traffic raw data shape is {df_traffic.shape}")
    
else:
    
    print("Skipping generation of incidents data")

Generating incidents csv
Getting head of incidents dataset
   incident_start_time incident_edge            incident_type  \
0                 7980    85756582#0  multi_vehicle_collision   
1                15805    85756582#0  multi_vehicle_collision   
2                23658    85756582#0  multi_vehicle_collision   
3                31472    85756582#0  multi_vehicle_collision   
4                39289    85756582#0  multi_vehicle_collision   

   accident_duration incident_lane  
0               4298             2  
1               5664  85756582#0_0  
2               2802  85756582#0_0  
3               4796  85756582#0_0  
4               6728  85756582#0_0  
Number of incidents in this simulation 165
Number of incidents in this simulation with less severe incidents 8
Number of incidents in this simulation with more severe incidents 157
Input traffic raw data shape is (2592000, 17)


In [10]:
df_traffic = df_traffic.drop(["vehicles_per_lane_1","vehicles_per_lane_0","lane_mean_speed_0","lane_mean_speed_1"],axis=1)

In [11]:
outgoing_edges = ["93676377#2-AddedOnRampEdge"]
incoming_edges = ["85756582#1-AddedOffRampEdge"]

In [12]:
edges = incoming_edges + outgoing_edges

In [13]:
# Apply rolling average to one sensor data : edges[0]
count = 0
df = df_traffic[df_traffic["identified_edge"] == edges[0] ]
df = df.reset_index(drop=True)
df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()

Y = df[["step","time_of_day","incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"]]
df_data = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
count+=1

In [14]:
print("Shape so far",df.shape) # df.shape[0] should match with number of timesteps

Shape so far (1296000, 16)


In [15]:
#Iterating over all other sensors and merging them
for index,edge in enumerate(edges):
    
    if index == 0:
        continue
    print(f"Processed {count}")
    df = df_traffic[df_traffic["identified_edge"] == edge ]
    df = df.reset_index(drop=True)
    df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
    
    df_temp = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
    df_data = dd.merge(left = df_data, right = df_temp , on = "step", how = "inner")

    count+=1
    

Processed 1


In [16]:
df_traffic_data = dd.merge(left = df_data, right = Y , on = "step", how = "inner")

In [17]:
print("Shape so far",df_traffic_data.shape)

Shape so far (1296000, 15)


In [18]:
df_vehicle = dd.read_csv(VEHICLE_DATASET,)

In [19]:
outgoing_edges = ["93676377#2-AddedOnRampEdge"]
incoming_edges = ["85756582#1-AddedOffRampEdge"]

In [20]:
# sensor details

sensors = {

    1: "93676377#2-AddedOnRampEdge",
    2:"85756582#1-AddedOffRampEdge"

}

In [21]:
junctions_sensor_combo ={
    2:[[1,2]]
}

In [22]:
interested_combo = junctions_sensor_combo[NUM_OF_JUNCTIONS]

In [23]:
print(interested_combo)
print(len(interested_combo))

[[1, 2]]
1


In [24]:
steps = [i for i in range(LENGTH_OF_SIM)]
data_dd = {"steps":steps}
dd_dataframe = dd.DataFrame(data_dd)
dd_dataframe.shape

(1296000, 1)

In [25]:
count = 1
for sensor_combination in interested_combo:
    print(count)
    outgoing_sensor = sensors[sensor_combination[0]]
    incoming_sensor = sensors[sensor_combination[1]]
    df_outgoing = df_vehicle[df_vehicle['identified_edge'] == outgoing_sensor]
    df_incoming = df_vehicle[df_vehicle['identified_edge'] == incoming_sensor]
    df_incoming = df_incoming.reset_index(drop=True)
    df_outgoing = df_outgoing.reset_index(drop=True)

    df_outgoing_unique = df_outgoing[df_outgoing["car_id"].notnull()]
    df_incoming_unique = df_incoming[df_incoming["car_id"].notnull()]

    df_outgoing_unique = df_outgoing_unique.groupby('car_id').first().reset_index()
    df_incoming_unique = df_incoming_unique.groupby('car_id').first().reset_index()

    df_outgoing_unique = df_outgoing_unique.sort_values(by = 'step')
    df_incoming_unique = df_incoming_unique.sort_values(by = 'step')
    
    df_merged = dd.merge(left = df_incoming_unique,right = df_outgoing_unique, on='car_id', how='inner')

    df_merged["travel_time"] = df_merged["step_x"] - df_merged["step_y"]

    df_travel_time_data = df_merged[["step_x","travel_time"]]
    df_travel_time_data = df_travel_time_data.groupby("step_x").mean().reset_index()
    
    df_travel_time_data2 = dd.merge(left = df_travel_time_data,right = dd_dataframe, right_on='steps',left_on="step_x", how='right')

    df_travel_time_data2[f"rolling_travel_time_{sensor_combination[0]}_{sensor_combination[1]}"] = df_travel_time_data2["travel_time"].rolling(300,min_periods=1).mean()

    df_travel_time_data2 = df_travel_time_data2.drop(["step_x","travel_time"],axis=1)
    df_traffic_data = dd.merge(left = df_traffic_data, right = df_travel_time_data2 , left_on = "step",right_on = "steps", how = "inner")
    df_traffic_data = df_traffic_data.drop(["steps"],axis=1)
    count+=1

1


In [26]:
df_traffic_data.head()

,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_1_2
0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,0,-1,False,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,0,-1,False,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,NaN,NaN,0,-1,False,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,0,NaN,NaN,0,-1,False,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0,NaN,NaN,0,-1,False,NaN


In [27]:
df_traffic_data.isna().sum()

step                                  0
rolling_junction_mean_speed_0       599
rolling_traffic_count_0             599
rolling_traffic_occupancy_0         599
rolling_junction_mean_speed_1       599
rolling_traffic_count_1             599
rolling_traffic_occupancy_1         599
time_of_day                           0
incident_edge                    506656
incident_start_time                   0
incident_type                    506997
accident_id                      506997
accident_duration                     0
incident_lane                         0
accident_label                        0
rolling_travel_time_1_2           36580
dtype: int64

In [28]:
df_traffic_data.to_csv(f"highway_processed_dataset.csv")

In [29]:
df_traffic_data.head()

,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_1_2
0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,0,-1,False,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,0,-1,False,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,NaN,NaN,0,-1,False,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,0,NaN,NaN,0,-1,False,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0,NaN,NaN,0,-1,False,NaN


In [30]:
df_traffic_data.shape

(1296000, 16)